# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [1]:
import math
import plotly.graph_objects as go
import simpy

# Import local libraries
from Tools.Check import Check
from Simulation import Simulation
from Microenvironment import Microenvironment
from Person import Person

In [2]:

periods = 60

simulation = Simulation()
simulation.run(periods)



Periods: 60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [3]:
time_axis = list(range(periods))
results = microenvironment.get_results()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=results , name='quanta concentration', line=dict(color='firebrick', width=1)))
fig.update_layout(title='Quanta concentration',
                   xaxis_title='Time (minutes)',
                   yaxis_title='Quanta concentration (quanta per m^-3)',
                   width=1280,
                   height=720)
fig.show()

NameError: name 'microenvironment' is not defined